In [2]:
# Import libraries 
import tensorflow as tf
from tensorflow.keras.layers import Input, LSTM, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.optimizers import Adam

from typing import Dict, List, Optional, Sequence, Tuple
import collections
import datetime
import glob
import numpy as np
import pathlib
import pandas as pd
import pretty_midi

from IPython import display
import matplotlib.pyplot as plt
import seaborn as sns

# data cleaning and preparation 

In [3]:
# Creating the required variables 
seed = 42
tf.random.set_seed(seed)
np.random.seed(seed)

# Sampling rate for audio playback 
_SAMPLING_RATE = 16000

In [8]:
# Setting the path 
data_fp = pathlib.Path("../../asset/maestro-v2.0.0/") # root data folder 
filenames = glob.glob(str(data_fp/"**/*.mid*")) # find all the midi files in the root folder
print("Number of files: ", len(filenames))

Number of files:  1282


In [11]:
# Test with a sample file 
sample_file = filenames[1]
print(sample_file)

pm = pretty_midi.PrettyMIDI(sample_file)

print("Number of instruments: ", len(pm.instruments))
instrument = pm.instruments[0]
instrument_name = pretty_midi.program_to_instrument_name(instrument.program)
print("Instrument name: ", instrument_name)

../../asset/maestro-v2.0.0/2013/ORIG-MIDI_03_7_6_13_Group__MID--AUDIO_09_R1_2013_wav--2.midi
Number of instruments:  1
Instrument name:  Acoustic Grand Piano


In [12]:
# Extract the notes 
for index, note in enumerate(instrument.notes[:10]): # look at the first 10 notes 
    note_name = pretty_midi.note_number_to_name(note.pitch)
    duration = note.end - note.start
    print(f"{index}: pitch={note.pitch}, note_name={note_name}, duration={duration:.4f}")

0: pitch=75, note_name=D#5, duration=0.0677
1: pitch=63, note_name=D#4, duration=0.0781
2: pitch=75, note_name=D#5, duration=0.0443
3: pitch=63, note_name=D#4, duration=0.0469
4: pitch=75, note_name=D#5, duration=0.0417
5: pitch=63, note_name=D#4, duration=0.0469
6: pitch=87, note_name=D#6, duration=0.0443
7: pitch=99, note_name=D#7, duration=0.0690
8: pitch=87, note_name=D#6, duration=0.0378
9: pitch=99, note_name=D#7, duration=0.0742


`pitch`: the perceptual quality of sound, determined by recognized by the MIDI note numbers. <br>
`step variable (index?)`: represents the time elapsed from the previous note or start of the track. <br>
`duration`: difference between the start time and end time of a note. <br>